In [ ]:
# ------------------- THIS CODE WAS RUN ON KAGGLE ------------------------- #

!pip install bertopic

  Obtaining dependency information for bertopic from https://files.pythonhosted.org/packages/43/da/182ef741d2510965a82a21332fa5706d1e9095c7a673436344442efceea9/bertopic-0.16.0-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 17.5 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for cython<3,>=0.27 from https://files.pythonhosted.org/packages/f8/26/ca0f1bb049b83c25cafa39f3fa5287c826a6ab36e665c906209e07f4deac/Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl.metadata
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl.metadata (3.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.

In [2]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
import json
import pandas as pd

import nltk
import gensim
import gensim.corpora as corpora
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from gensim.models.coherencemodel import CoherenceModel

In [3]:
# df=pd.read_pickle("/kaggle/input/bertopic-bug-data-preprocessed/preprocessed_bug_data_for_bertopic.pkl")
# df=pd.read_pickle("/kaggle/input/previous-preprocessed/preprocessed_bug_data.pkl")
df=pd.read_pickle("/kaggle/input/preprocessed-bug-data-for-bertopic/preprocessed_bug_data_for_bertopic.pkl")

df.head()
df.columns

Index(['title', 'description', 'comments', 'unique_id'], dtype='object')

In [4]:
df['title_description'] = df['title'] + ' ' + df['description']
df['title_description_comments'] = df['title'] + ' ' + df['description'] + ' ' + df['comments'].fillna('')
df.columns

Index(['title', 'description', 'comments', 'unique_id', 'title_description',
       'title_description_comments'],
      dtype='object')

In [5]:
len(df)

10538

In [6]:
# df = df[df['title_description_comments'].apply(lambda x: len(x.split()) >= 4)]

In [7]:
len(df)

10538

In [8]:
new_df = df['title_description_comments']
new_df.head()

0    Bitcode disable decreasing score disable bitco...
1    FEATURE rescan finished remove rescan 1 argume...
2    FEATURE Update docker tagged build workflow fi...
3    Update python support 3 11 version  updated se...
4    Dynamic Analyser iOS support Dynamic Analyser ...
Name: title_description_comments, dtype: object

In [9]:
len(new_df)

10538

In [32]:
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic.vectorizers import ClassTfidfTransformer
# from sklearn.decomposition import PCA
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
umap_model = UMAP(n_neighbors=10, n_components=15, min_dist=0.000001)
ctfidf_model = ClassTfidfTransformer()

hdbscan_model = HDBSCAN(min_samples=40, prediction_data=True, gen_min_span_tree=True)

In [33]:
embedding_model=SentenceTransformer('all-MiniLM-L6-v2')
vectorizer_model = CountVectorizer(stop_words="english")
# pca_model = PCA(n_components=30)
model = BERTopic(umap_model=umap_model,hdbscan_model=hdbscan_model,ctfidf_model=ctfidf_model, embedding_model=embedding_model,vectorizer_model=vectorizer_model,top_n_words=7,language="english",calculate_probabilities=True, verbose=True)

In [34]:
topics, prob = model.fit_transform(new_df)

2023-12-18 00:34:13,524 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/330 [00:00<?, ?it/s]

2023-12-18 00:34:28,568 - BERTopic - Embedding - Completed ✓
2023-12-18 00:34:28,569 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2023-12-18 00:34:37,090 - BERTopic - Dimensionality - Completed ✓
2023-12-18 00:34:37,092 - BERTopic - Cluster - Start clustering the reduced embeddings
2023-12-18 00:34:39,043 - BERTopic - Cluster - Completed ✓
2023-12-18 00:34:39,050 - BERTopic - Representation - Extracting topics from clusters using representation models.
2023-12-18 00:34:41,174 - BERTopic - Representation - Completed ✓


In [35]:
model.visualize_barchart(top_n_topics=10, n_words=9)

In [36]:
model.visualize_topics()

In [37]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1828,-1_semgrep_file_like_issue,"[semgrep, file, like, issue, rule, bug, use]",[suppress nosemgrep autoformatter ’ use nosemg...
1,0,2780,0_psalm_type_snippet_summary,"[psalm, type, snippet, summary, array, php, er...",[Typing dependency injection container Hello w...
2,1,2487,1_pylint_behavior_python_bitbucket,"[pylint, behavior, python, bitbucket, output, ...",[Pylint 2 10 0 incorrectly assumes HOME cache ...
3,2,726,2_golangci_lint_summary_linter,"[golangci, lint, summary, linter, yes, issue, ...",[linters setting thelper test name working Wel...
4,3,633,3_java_pmd_rule_maven,"[java, pmd, rule, maven, class, org, gradle]",[java MissingOverrideRule exception analyzing ...
5,4,368,4_rule_semgrep_file_json,"[rule, semgrep, file, json, output, issue, like]",[wait end flush output file feature request re...
6,5,257,5_mobsf_android_apk_info,"[mobsf, android, apk, info, analysis, u0000, s...",[program killed reason specific apk file ENVIR...
7,6,174,6_ruby_gem_brakeman_rails,"[ruby, gem, brakeman, rails, bundler, rb, dawn]",[superclass method process attrasgn brakeman r...
8,7,166,7_typescript_javascript_pattern_semgrep,"[typescript, javascript, pattern, semgrep, mat...",[TypeScript import pattern matched named place...
9,8,161,8_compiler_es6_closure_goog,"[compiler, es6, closure, goog, jscomp, functio...",[Type deducted goog bind used Consider followi...


In [38]:
len(topics)

10538

In [39]:
topics[0]

-1

In [40]:
model.visualize_heatmap()

In [41]:
model.visualize_hierarchy()

In [42]:
model.get_representative_docs(4)

['wait end flush output file feature request related problem Please describe running scan taking several hour complete wish result work running also afraid crash point 2 3 hour going lose everything semgrep might found Describe solution like scan running sitting 14 22 rule done love result 14 rule finished scanning already output file Describe alternative considered Run semgrep per rule Additional context N thanks see making wa surprised issue already Seems like obvious change suspect plan already Part reason might expect semgrep fast want get faster corpus running public chance anything share major perf initiative underway would love get representative sample added benchmarking suite think might already happened Gitlab cc wa running ruleset Cc issue marked activity 30 day Please leave comment think issue still relevant prioritized otherwise automatically closed 7 day always reopen later would like understand pressing issue assume much easier moved functionality semgrep core still capt

In [43]:
import pandas

In [44]:
final_df = pd.DataFrame({"topic": topics,"title_description_comments": df['title_description_comments'], "unique_id": df['unique_id']})
final_df

,topic,title_description_comments,unique_id
0,-1,Bitcode disable decreasing score disable bitco...,ID_2
1,-1,FEATURE rescan finished remove rescan 1 argume...,ID_3
2,-1,FEATURE Update docker tagged build workflow fi...,ID_4
3,9,Update python support 3 11 version updated se...,ID_5
4,5,Dynamic Analyser iOS support Dynamic Analyser ...,ID_6
...,...,...,...
10647,2,Comments issue golint ignored default golint a...,ID_10649
10648,6,gem install codesake dawn error Trying install...,ID_10650
10649,6,Rake install issue private ey Hi whilst trying...,ID_10651
10650,-1,DataContractJsonSerializer vulnerable type dat...,ID_10652


In [52]:
final_df.to_csv('/kaggle/working/final_df3.csv',index=False) # save to notebook output
final_df.head(10)

,topic,title_description_comments,unique_id
0,-1,Bitcode disable decreasing score disable bitco...,ID_2
1,-1,FEATURE rescan finished remove rescan 1 argume...,ID_3
2,-1,FEATURE Update docker tagged build workflow fi...,ID_4
3,9,Update python support 3 11 version updated se...,ID_5
4,5,Dynamic Analyser iOS support Dynamic Analyser ...,ID_6
5,-1,Uninstall Steps Needed installed MobSF without...,ID_7
6,5,FEATURE Support non default XMLNS attribute an...,ID_8
7,4,Include new Semgrep rule semgrep rule android ...,ID_9
8,5,FEATURE Optimizing Storage Privileges Database...,ID_10
9,-1,FEATURE Enhancing secret detection regex rule ...,ID_11


In [49]:
documents = pd.DataFrame({"Document": new_df,
                          "ID": range(len(new_df)),
                          "Topic": topics})
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = model._preprocess_text(documents_per_topic.Document.values)

# Extract vectorizer and analyzer from BERTopic
vectorizer = model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names_out()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
coherence = coherence_model.get_coherence()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [50]:
coherence

0.6703151984164809